In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel(f'./file/3.model.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['Year'] = input_['Year'].astype(int)
input_['Year Code'] = input_['Year Code'].astype(int)
input_['Make Code'] = input_['Make Code'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =


work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        try:
            b = 0
            while True:
                b += 1
                
                try:
                    resp = requests.get(f'''https://www.auveco.com/store_ss2/extensions/UnlockCommerce/YearMakeModel/1.0.0/Modules/YearMakeModel/SuiteScript2/YearMakeModel.Service.ss?action=listbyymm&limit=100&make={input_.loc[a, 'Make']}&model={input_.loc[a, 'Model']}&offset=0&page=1&sort=relevance%3Adesc&year={input_.loc[a, 'Year']}''',
                                        headers=get_header(),
                                        proxies=get_proxy(),
                                        timeout=(10, 10))

                    if resp.status_code == 200:
                        json_ = resp.json()

                        if json_['success'] == 'OK':
                            break
                except KeyboardInterrupt:
                    break
                except:
                    continue

            # = = = = = = = = = = = = = = =

            if json_['totalRecords'] <= 0 and json_['totalPages'] <= 0:
                raise

            # = = = = = = = = = = = = = = =

            if json_['totalRecords'] > 0:
                df_temp = pd.DataFrame({'Make': input_.loc[a, 'Make'],
                                        'Model': input_.loc[a, 'Model'],
                                        'Year': input_.loc[a, 'Year'],
                                        'SKU Count': json_['totalRecords'],
                                        'Page': [i+1 for i in range(json_['totalPages'])]})
                
                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            crawler_status = 'ok'
        except:
            output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        if work.qsize() % 1_000 == 0 and work.qsize() != 0:
            clear_output()

        print(f'''[状态：{crawler_status}，尝试次数：{b}] - {input_.loc[a, 'Make']} > {input_.loc[a, 'Model']} > {input_.loc[a, 'Year']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(30):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['Make', 'Model', 'Year', 'Page'],
                                                ascending=[True, True, False, True],
                                                ignore_index=True)
    output_correct.to_excel(f'''./file/4.page-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if not output_error.empty:
    output_error = output_error.sort_values(by=['Make', 'Model', 'Year'],
                                            ascending=[True, True, False],
                                            ignore_index=True)
    output_error.to_excel(f'''./file/page_error.xlsx''', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

[状态：ok，尝试次数：23] - TOYOTA > Solara > 2002
[剩余数量：1000] - [当前时间：17:38:55]

[状态：ok，尝试次数：18] - TOYOTA > Solara > 2001
[剩余数量：999] - [当前时间：17:38:55]

[状态：ok，尝试次数：1] - TOYOTA > Supra > 1998
[剩余数量：998] - [当前时间：17:38:55]

[状态：ok，尝试次数：14] - TOYOTA > Solara > 1999
[剩余数量：997] - [当前时间：17:38:55]

[状态：ok，尝试次数：22] - TOYOTA > Starlet > 1981
[剩余数量：996] - [当前时间：17:38:56]

[状态：ok，尝试次数：1] - TOYOTA > Supra > 1994
[剩余数量：995] - [当前时间：17:38:56]

[状态：ok，尝试次数：5] - TOYOTA > Supra > 1987
[剩余数量：994] - [当前时间：17:38:57]

[状态：ok，尝试次数：1] - TOYOTA > Solara > 2000
[剩余数量：993] - [当前时间：17:38:57]

[状态：ok，尝试次数：28] - TOYOTA > Supra > 1997
[剩余数量：992] - [当前时间：17:38:57]

[状态：ok，尝试次数：5] - TOYOTA > Supra > 1985
[剩余数量：991] - [当前时间：17:38:57]

[状态：ok，尝试次数：2] - TOYOTA > Supra > 1995
[剩余数量：990] - [当前时间：17:38:58]

[状态：ok，尝试次数：32] - TOYOTA > Supra > 1989
[剩余数量：989] - [当前时间：17:38:58]

[状态：ok，尝试次数：16] - TOYOTA > Supra > 1992
[剩余数量：988] - [当前时间：17:38:58]

[状态：ok，尝试次数：42] - TOYOTA > Starlet > 1982
[剩余数量：987] - [当前时间：17:38:58]

[状态：ok，尝试次数：37] -